# Cahier des charges - Projet Green Graph
*Equipe CesiCDP - Chef de projet : Leila | Opérateurs : Tom, Edwin*</br>
## Sommaire :
    1. Contexte
    2. Problématique
    3. Contraintes
    4. Lexique


### 1. Contexte

L'ADEME cherche à optimiser la tournée de ses transports en commun afin de réduire ses coûts financiers et son bilan carbone.

### 2. Problématique
Comment résoudre un problème de logistique s’apparentant au problème du voyageur de commerce, prenant en compte des contraintes supplémentaires ?


### 3. Contraites
| Importance | Description |
| ----------- | ----------- |
| Haute | Réduire au maximum la pollution émise par les véhicules |
| Haute | Optimiser les coûts |

### 4. Lexique

### 5. Données de Test
Pour répondre à ce probleme nous utiliserons les villes de france obtenu grace à World City DB , 

In [1]:
import requests
import json
from geopy.geocoders import Nominatim

def geocode_ville(nom_ville):
    """Convertit le nom d'une ville en coordonnées (latitude, longitude)"""
    geolocator = Nominatim(user_agent="routing_app")
    location = geolocator.geocode(nom_ville)
    
    if location:
        return (location.latitude, location.longitude)
    else:
        raise ValueError(f"Impossible de trouver les coordonnées pour {nom_ville}")

def calculer_temps_trajet(ville_depart, ville_arrivee, mode="driving"):
    """
    Calcule le temps de trajet entre deux villes en utilisant l'API OSRM
    
    Args:
        ville_depart (str): Nom de la ville de départ
        ville_arrivee (str): Nom de la ville d'arrivée
        mode (str): Mode de transport (driving, cycling, walking)
        
    Returns:
        tuple: (temps en secondes, distance en mètres, temps formaté)
    """
    # Obtenir les coordonnées des villes
    try:
        lat1, lon1 = geocode_ville(ville_depart)
        lat2, lon2 = geocode_ville(ville_arrivee)
    except ValueError as e:
        return (None, None, str(e))
    
    # Construction de l'URL pour l'API OSRM
    url = f"http://router.project-osrm.org/route/v1/{mode}/{lon1},{lat1};{lon2},{lat2}"
    params = {
        "overview": "false",
        "alternatives": "false",
    }
    
    # Appel à l'API OSRM
    response = requests.get(url, params=params)
    
    if response.status_code != 200:
        return (None, None, f"Erreur lors de l'appel à l'API: {response.status_code}")
    
    data = response.json()
    
    if data["code"] != "Ok":
        return (None, None, f"Erreur de l'API OSRM: {data['code']}")
    
    # Extraction des informations de temps et de distance
    route = data["routes"][0]
    duree_secondes = route["duration"]
    distance_metres = route["distance"]
    
    # Formatage du temps pour l'affichage
    heures, reste = divmod(duree_secondes, 3600)
    minutes, secondes = divmod(reste, 60)
    
    temps_formate = ""
    if heures > 0:
        temps_formate += f"{int(heures)} heure{'s' if heures > 1 else ''} "
    if minutes > 0:
        temps_formate += f"{int(minutes)} minute{'s' if minutes > 1 else ''}"
    
    return (duree_secondes, distance_metres, temps_formate.strip())

def afficher_itineraire(ville_depart, ville_arrivee, mode="driving"):
    """Affiche les informations d'itinéraire entre deux villes"""
    modes = {
        "driving": "en voiture",
        "cycling": "à vélo",
        "walking": "à pied"
    }
    
    duree, distance, message = calculer_temps_trajet(ville_depart, ville_arrivee, mode)
    
    if duree is None:
        print(message)
        return
    
    print(f"Itinéraire de {ville_depart} à {ville_arrivee} {modes.get(mode, '')}:")
    print(f"Temps de trajet: {message}")
    print(f"Distance: {distance/1000:.1f} km")


In [4]:
print(afficher_itineraire("Reims", "Paris", "driving"))

Itinéraire de Reims à Paris en voiture:
Temps de trajet: 1 heure 40 minutes
Distance: 144.0 km
None


In [3]:
def decimal_to_hms(decimal_hours):
    hours = int(decimal_hours)
    minutes_float = (decimal_hours - hours) * 60
    minutes = int(minutes_float)
    seconds = round((minutes_float - minutes) * 60)
    return hours, minutes, seconds